**Search Agent**

For the given Morocco Map heuristics function try to come up with other heuristics and see what s their performance on the search agent.

In [ ]:
import math
import heapq  


morocco_map = {
    'Casablanca': ['Rabat', 'El Jadida'],
    'Rabat':      ['Casablanca', 'Meknes', 'Tangier'],
    'El Jadida':  ['Casablanca', 'Marrakech'],
    'Meknes':     ['Rabat', 'Fez', 'Ifrane'],
    'Tangier':    ['Rabat'],
    'Marrakech':  ['El Jadida'],
    'Fez':        ['Meknes', 'Ifrane'],
    'Ifrane':     ['Meknes', 'Fez']
}


# Tangier is North (High Y) and Marrakech is South (Low Y)
city_coords = {
    'Tangier':    (1, 10),
    'Rabat':      (0.5, 8),
    'Casablanca': (0, 6),
    'El Jadida':  (-0.5, 4),
    'Marrakech':  (-0.5, 0),
    'Meknes':     (2, 7),
    'Fez':        (3, 7.5),
    'Ifrane':     (2.5, 5)
}

In [ ]:
def get_heuristic(city, goal, method='euclidean'):
    x1, y1 = city_coords[city]
    x2, y2 = city_coords[goal]
    
    if method == 'manhattan':
        # You can only move along grid lines (Up/Down/Left/Right)
        return abs(x1 - x2) + abs(y1 - y2)
        
    elif method == 'euclidean':
        # Diagonal movement allowed
        return math.sqrt((x1 - x2)**2 + (y1 - y2)**2)
    #TASK: Add more heuristics here 

    else:
        return 0 # If heuristic is 0, A* turns back into BFS

In [ ]:
def a_star_agent(graph, start, goal, heuristic_type='euclidean'):
    # We store tuples: (Score, Current_City, Path_List)
    # The heap automatically sorts by the first item (Score)
    priority_queue = []
    
    # Calculate initial score (0 cost + estimated distance)
    initial_h = get_heuristic(start, goal, heuristic_type)
    heapq.heappush(priority_queue, (initial_h, start, [start]))
    
    # To keep track of the lowest cost to reach a node so far
    costs_so_far = {start: 0}

    print(f"Starting A* Search using {heuristic_type}")

    while priority_queue:
        # 1. Get the path with the LOWEST Score
        current_score, current_city, path = heapq.heappop(priority_queue)
        
        # 2. Check Goal
        if current_city == goal:
            return path
        
        # 3. Explore Neighbors
        for neighbor in graph[current_city]:
            # CALCULATE G: Cost to get here + 1 hop to neighbor
            new_cost = costs_so_far[current_city] + 1 
            
            # If we haven't visited this neighbor, OR we found a faster way to it
            if neighbor not in costs_so_far or new_cost < costs_so_far[neighbor]:
                costs_so_far[neighbor] = new_cost
                
                # CALCULATE H: Estimate from neighbor to goal
                heuristic = get_heuristic(neighbor, goal, heuristic_type)
                
                # CALCULATE F: Total Score
                final_score = new_cost + heuristic
                
                # Add to fringe
                new_path = list(path)
                new_path.append(neighbor)
                heapq.heappush(priority_queue, (final_score, neighbor, new_path))
                
                print(f"  Checked {neighbor}: Cost(g)={new_cost}, Guess(h)={heuristic:.1f}, Score(f)={final_score:.1f}")

    return "No path found"

In [ ]:
#Use this fuction with your created heuristics to find the best path from Meknes to Casablanca
print("A* Route (Euclidean):", a_star_agent(morocco_map, 'Meknes', 'Casablanca', ''))


You want to see how messy an 8-Puzzle board is

Look at the board.

For each tile (1-8), calculate how far it is from where it belongs.

Sum these numbers up to get a "Messiness Score."

In [ ]:
# CALCULATE THE "MESSINESS" SCORE

# THE GOAL: This is where every tile WANTS to be.
# We use a dictionary to look up coordinates: Number : (Row, Col)
# Example: Tile '1' wants to be at Row 0, Col 1
goal_positions = {
    0: (0, 0), 1: (0, 1), 2: (0, 2),
    3: (1, 0), 4: (1, 1), 5: (1, 2),
    6: (2, 0), 7: (2, 1), 8: (2, 2)
}

def get_manhattan_score(current_board):
    total_distance = 0
    
    # Loop through the 9 spots on the board (0 to 8)
    for i in range(9):
        # Get the tile number at this spot
        tile = current_board[i]
        
        # We skip the empty space (0) because it doesn't count
        if tile != 0:
            # The position of the tile (Current Row, Col)
            current_row = i // 3  # Math trick to get Row
            current_col = i % 3   # Math trick to get Column
            
            # The goal where the tile wants to be
            target_row, target_col = goal_positions[tile]
            
            # TASK: CALCULATE DISTANCE
            # Distance = difference in rows + difference in cols
            # Hint: Use abs() to keep numbers positive!
            
            dist = abs(current_row - target_row) + abs(current_col - target_col)
            total_distance = total_distance + dist
            
    return total_distance

# TEST YOUR CODE (Do not change)
# This board has:
# - '1' moved 1 step down (needs 1 step up)
# - '4' moved 1 step up (needs 1 step down)
# Total Score should be 2.
test_board = [
    4, 2, 3,  # 0, 1, 2
    1, 5, 6,  # 3, 4, 5
    7, 8, 0   # 6, 7, 8
]

score = get_manhattan_score(test_board)
print(f"Your Calculated Score: {score}")

if score == 2:
    print("The AI now knows exactly how messy the board is.")
else:
    print(f"Try again. Expected 2, but got {score}.")

**Adverserial Agent**

You are playing the Coin Game. There is a pile of coins. On your turn, you can take 1, 2, or 3 coins.

Goal: You want to take the last coin to win.
Your Job: Write an agent that looks at the pile and checks: If I take X coins, do I win immediately.

Check if taking 1 coin leaves 0 remaining.

Check if taking 2 coins leaves 0 remaining.

Check if taking 3 coins leaves 0 remaining.

If none of them win, just take 1 (play it safe).

In [ ]:
def get_winning_move(coins_in_pile):
    print(f"Thinking about pile size: {coins_in_pile}...")
    
    # OPTION A: What happens if I take 1?
    remaining = coins_in_pile - 1
    if remaining == 0:
        return 1
        
    # TASK: CHECK IF TAKING 2 WINS 
    remaining = coins_in_pile - 2
    if remaining == 0:
        return 2 

    # TASK: CHECK IF TAKING 3 WINS 
    remaining = coins_in_pile - 3
    if remaining == 0:
        return 3 

    # Taking 1 if you can t win
    return 1

# TEST YOUR BOT 
# Scenario: There are 3 coins left. The AI should notice it can take all 3.
test_pile = 3

move = get_winning_move(test_pile)
print(f"Bot chose to take: {move}")

if move == 3:
    print("The Agent saw the win and took it.")
elif move == 1:
    print("The Agent played it safe instead of winning.")

Change the rules order and see how the model will behave differently and how the output will change each time (You can also add some rules I want to see your creativity in your approaches!).

In [ ]:
# THE BOARD: A simple list of 9 items
board = [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']

# 1. SIMPLE WIN CHECKER
# Instead of complex loops, we just list the 8 ways to win
def check_winner(b, player):
    # Check Rows
    if b[0] == player and b[1] == player and b[2] == player: return True
    if b[3] == player and b[4] == player and b[5] == player: return True
    if b[6] == player and b[7] == player and b[8] == player: return True
    # Check Columns
    if b[0] == player and b[3] == player and b[6] == player: return True
    if b[1] == player and b[4] == player and b[7] == player: return True
    if b[2] == player and b[5] == player and b[8] == player: return True
    # Check Diagonals
    if b[0] == player and b[4] == player and b[8] == player: return True
    if b[2] == player and b[4] == player and b[6] == player: return True
    return False


def get_best_move(b):
    #TASK: Play with the rules and change their order to make the Agent more aggressive or defensive.
    # RULE 1: Can I (AI 'O') win right now?
    for i in range(9):
        if b[i] == ' ':            # If spot is empty
            b[i] = 'O'             # Try moving there
            if check_winner(b, 'O'):
                return i           # FOUND A WINNING MOVE!
            b[i] = ' '             # Reset spot (Undo)

    # RULE 2: Will the Human ('X') win next?
    for i in range(9):
        if b[i] == ' ':
            b[i] = 'X'             # Pretend to be the Human
            if check_winner(b, 'X'):
                b[i] = 'O'         # Block them!
                return i
            b[i] = ' '             # Reset

    # RULE 3: Just take the center if open (It's the best spot)
    if b[4] == ' ':
        return 4

    # RULE 4: Pick the first open corner
    for i in [0, 2, 6, 8]:
        if b[i] == ' ':
            return i

    # RULE 5: Pick whatever is left
    for i in range(9):
        if b[i] == ' ':
            return i

# Main Game Loop
print("--- Simple Tic-Tac-Toe ---")
print("You are X. The AI is O.")
print(" 0 | 1 | 2 ")
print("---+---+---")
print(" 3 | 4 | 5 ")
print("---+---+---")
print(" 6 | 7 | 8 \n")

while True:
    # 1. Human Move
    move = int(input("Enter position (0-8): "))
    if board[move] != ' ':
        print("Occupied! Try again.")
        continue
    board[move] = 'X'

    # Check if Human Won
    if check_winner(board, 'X'):
        print("You Win!")
        break

    # Check for Tie (Board full)
    if ' ' not in board:
        print("It's a Tie!")
        break

    # 2. AI Move
    ai_move = get_best_move(board)
    board[ai_move] = 'O'
    
    # Print Board
    print(f"\nAI chose {ai_move}:")
    print(f" {board[0]} | {board[1]} | {board[2]} ")
    print(f" {board[3]} | {board[4]} | {board[5]} ")
    print(f" {board[6]} | {board[7]} | {board[8]} ")

    if check_winner(board, 'O'):
        print("AI Wins!")
        break